In [ ]:
#導入資料+資料預處理
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
x=pd.read_csv("test_users.csv")
y=pd.read_csv("train_users_2.csv")
submit = pd.read_csv("sample_submission_NDF.csv")
x.info() 
y.info()
x.isnull().sum()
y.isnull().sum()

In [ ]:
#補值+合併資料表
data = y.append(x)
data
data.reset_index(inplace=True, drop=True)
data['age'] = data['age']. fillna(data['age'].mean( ))

In [ ]:
#探索資料
sns.countplot(data['country_destination'])
sns.countplot(data['language'],hue=data ['country_destination'])
plt.show()

In [ ]:
sns.countplot(data['country_destination'])
sns.countplot(data['gender'],hue=data ['country_destination'])
plt.show()

In [ ]:
g = sns.FacetGrid(data,col='country_destination') 
g.map(sns.distplot,'age',kde=False)
plt.show()

In [ ]:
sns.countplot(data['country_destination'])
sns.countplot(data['affiliate_provider'],hue=data ['country_destination'])
plt.show()

In [ ]:
sns.countplot(data['country_destination'])
sns.countplot(data['signup_app'],hue=data ['country_destination'])
plt.show()

In [ ]:
#將類別資料轉成整數
data['signup_app'] = data['signup_app'].astype('category').cat.codes
data ['affiliate_provider'] = data['affiliate_provider'].astype('category').cat.codes
data['language'] = data['language'].astype('category').cat.codes
data['gender'] = data['gender'].astype('category').cat.codes
data['country_destination'] = data['country_destination'].astype('category').cat.codes

In [ ]:
#留下需要的特徵
dataTrain = data[pd.notnull(data['country_destination'])].sort_values(by=["id"])
dataTest = data[~pd.notnull(data['country_destination'])].sort_values(by=["id"])
dataTrain.columns
dataTrain = dataTrain[['country_destination', 'language',"gender", 'age','affiliate_provider','signup_app']]
dataTest = dataTest[['language',"gender", 'age','affiliate_provider','signup_app']]
dataTrain

In [ ]:
#利⽤隨機森林來預測去的國家
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(criterion='gini'
                            ,n_estimators=1000
                            ,min_samples_split=12
                            ,min_samples_leaf=1
                            ,oob_score=True
                            ,random_state=1
                            ,n_jobs=-1)
rf.fit(dataTrain.iloc[:,1:],dataTrain.iloc [:,0])
print("%.4f" & rf.oob_score_)

In [ ]:
#產⽣ Submit 檔
pd.concat((pd.DataFrame(dataTrain.iloc[:,1:].columns,columns = ['variable']),
           pd.DataFrame(rf.feature_importances_,columns = ['importance'])),
          axis = 1).sort_values (by='importance', ascending = False) [:20]
rf_res = rf.predict(dataTest)
submit['country_destination'] = rf_res
submit['country_destination'] = submit['country_destination'].astype(str)
submit.to_csv('submit_NDF.csv',index=False)